In [16]:
import cv2
import numpy as np
import pytesseract

In [17]:
def detectar_matricula(imatge_path):
    imatge = cv2.imread(imatge_path)

    #grisos
    grisos = cv2.cvtColor(imatge,cv2.COLOR_BGR2GRAY)

    #Top_hat
    kernel = np.ones((15,15), np.uint8)
    tophat = cv2.morphologyEx(grisos, cv2.MORPH_TOPHAT, kernel)

    #Binaritzem
    _, threshold = cv2.threshold(tophat,50,255, cv2.THRESH_BINARY)

    #Trobem contorns
    contorns, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    #Establim mides matrícula ???

    min_area = 500
    max_area = 2500
    aspect_ratio = 4
    plate_contours = []

    for contorn in contorns:
        x,y,z,t = cv2.boundingRect(contorn)
        area = cv2.contourArea(contorn)
        aspect = z / float(t)

        if min_area < area < max_area and aspect > aspect_ratio:
            plate_contours.append(contorn)

    #Ho dibuixem
    for contorn in plate_contours:
        x,y,z,t = cv2.boundingRect(contorn)
        cv2.rectangle(imatge, (x,y), (x+z,y+t), (0, 255, 0), 2)
        matricula = grisos[y:y+t, x:x+z]
        matricula_eq = cv2.equalizeHist(matricula)
        text = pytesseract.image_to_string(matricula_eq, lang='eng', config='--psm 7')
        print('Matrícula:', text)

    #Plotegem
    cv2.imshow('Matricules detectades', imatge)
    cv2.waitKey(0)
    cv2.destroyAllWindows()



In [18]:
detectar_matricula('fotos/cotxe1.jpg')

TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.